In [1]:
# Import libraries for data handling, image processing, and SVM classification
import os
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from PIL import Image


In [2]:
# Define the target image size and a helper function to load and flatten images
IMG_SIZE = (64, 64)  # Small size to reduce feature dimensionality

def load_and_flatten(image_path):
    img = Image.open(image_path).convert('RGB')  # Use grayscale with 'L' if needed
    img = img.resize(IMG_SIZE)
    arr = np.array(img)
    return arr.flatten()


In [3]:
# Load CSV files and add full paths to image filenames
train_df = pd.read_csv("../data/train.csv")
val_df = pd.read_csv("../data/validation.csv")

train_df["image_id"] = train_df["image_id"].apply(lambda x: os.path.join("../data/train", f"{x}.png"))
val_df["image_id"] = val_df["image_id"].apply(lambda x: os.path.join("../data/validation", f"{x}.png"))


In [4]:
# Convert images to flattened numpy arrays (1D vectors per image)
X_train = np.array([load_and_flatten(p) for p in train_df["image_id"]])
y_train = train_df["label"].values

X_val = np.array([load_and_flatten(p) for p in val_df["image_id"]])
y_val = val_df["label"].values


In [5]:
# Train a linear SVM on the image vectors and evaluate on validation data
svm = SVC(kernel='rbf') # linear - 0.4224
svm.fit(X_train, y_train)

val_preds = svm.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, val_preds))


Validation Accuracy: 0.6304


In [6]:
# Optional: load test set and make predictions
test_df = pd.read_csv("../data/test.csv")
test_df["image_id"] = test_df["image_id"].apply(lambda x: os.path.join("../data/test", f"{x}.png"))

X_test = np.array([load_and_flatten(p) for p in test_df["image_id"]])
preds = svm.predict(X_test)

# Save predictions to CSV
submission = pd.DataFrame({
    "image_id": test_df["image_id"].apply(lambda x: os.path.splitext(os.path.basename(x))[0]),
    "label": preds
})
submission.to_csv("submission_svm.csv", index=False)

